# Generate `nbgitpuller` links for your JupyterHub

When users click an `nbgitpuller` link pointing to your JupyterHub,

1. They are asked to log in to the JupyterHub if they have not already
2. The git repository referred to in the nbgitpuller link is made up to date in their home directory (keeping local changes if there are merge conflicts)
3. They are shown the specific notebook / directory referred to in the nbgitpuller link.

This is a great way to distribute materials to students.

# Generate `nbgitpuller` links for your JupyterHub

## Sequence of events when users click an `nbgitpuller` link pointing to your JupyterHub,

1. They are asked to log in to the JupyterHub if they have not already
2. The git repository referred to in the nbgitpuller link is made up to date in their home directory (keeping local changes if there are merge conflicts)
3. They are shown the specific notebook / directory referred to in the nbgitpuller link.

This is a great way to distribute materials to students.

## Canvas LMS: Assignment Links vs Custom Fields

The Canvas LMS expects the assignment link to include URL encoded parameters since the request is sent to the External Tool as a POST request (in this case IllumiDesk is the External Tool). However, all characters (even those considered safe) after the domain and `next=` part should be URL encoded, such as the `/`, `&`, and `=` characters.

The `Custom Fields` text box in the App -> Settings section, on the other hand, does not expect all characters to be URL encoded. The `/` characters that are assigned as part of the query parameter values should be encoded, but not the `&` and `=` characters.

## Usage

- **Assignment Link**: creates a string value which represents an `Assignment` link by toggling the check box next to the `is_assignment_link` label. If unchecked, the tool will create a string to add to the Custom Field section.
- **Jupyter Lab Link**: creates a string value which redirects the user to a `Jupyter Lab` workspace instead of the `Jupyter Classic` workspace.
- **LTI Launches**: adds the route associated to the LTI 1.1 login handler. If disabled, it is assumed that the user is using the default authentication class bound to the root of the `domain_url` value.
- **Default Values**: to avoid having to enter the same values in the widget's text fields on a repetitive basis, add the string values to the function's parameters. For example, the `branch` parameter defaults to `master`.

In [6]:
import os
from ipywidgets import interact
from urllib.parse import urlunparse, urlparse, urlencode, parse_qs, parse_qsl, quote
from IPython.display import Markdown


@interact
def make_launch_link(is_assignment_link=True, is_jupyterlab=True, is_lti11=True, branch='master', hub_url='https://my.hub.com', repo_url='', urlpath=''):
    """
    Generate a launch request which clones and merges source files from a git-based
    repository.

    Args:
      is_assignment_link (bool): set to True to create a full assignment link, defaults to True.
      is_jupyterlab (bool): set to True to launch Jupyter Lab workspaces, defaults to True.
      is_lti11 (bool): set to True to initiate launch requests with the LTI 1.1 standard.
      branch (str): git repo branch
      hub_url (str): full hub url which needs to include scheme (http or https) and netloc (full domain).
      repo_url (str): full git repo url which needs to include scheme (http or https), netloc (full domain) and path.
      url_path (str): a path to redirect users to after the workspace has successfully spawned (started).

    Returns:
      An interactive IPython.display.Markdown object.
    """

    # Parse the query to its constituent parts
    domain_scheme, domain_netloc, domain_path, domain_params, domain_query_str, domain_fragment = urlparse(hub_url.strip())
    
    repo_scheme, repo_netloc, repo_path, repo_params, repo_query_str, repo_fragment = urlparse(repo_url.strip())
    folder_from_repo_url_path = os.path.basename(os.path.normpath(repo_path))
    
    # Make sure the path doesn't contain multiple slashes
    if not domain_path.endswith('/'):
        domain_path += '/'
    domain_path += 'user-redirect/git-pull'
    
    # With Canvas using LTI 11 Assignment launch requests all characters after the netloc are considered unsafe.
    # When adding custom parameters within the App Settings -> Custom Fields section, only items after the 
    path_encoded = ''
    if is_assignment_link:
        path_encoded = quote(domain_path, safe='')
    else:
        path_encoded = quote(domain_path)

    path_redirect_url = f'next={path_encoded}'
    if is_lti11:
        assignment_link_path = f'/hub/lti/launch?next={path_encoded}'
    else:
        assignment_link_path = f'/hub?next={path_encoded}'
    
    # Create a tuple of query params from original domain link
    query_params_from_hub_url = parse_qsl(domain_query_str, keep_blank_values=True)
    
    # Set path based on whether or not the user would like to spawn JupyterLab or Jupyter Classic
    urlpath_workspace = ''
    if is_jupyterlab:
        urlpath_workspace = f'lab/tree/{folder_from_repo_url_path}/{urlpath}?autodecode'
    else:
        urlpath_workspace = f'tree/{folder_from_repo_url_path}/{urlpath}'
    
    # Create a tuple of query params for git functionality. Check whether or not we want to launch with
    # jupyterlab to add additional items to the path.
    query_params_for_git = [('repo', repo_url), ('branch', branch), ('urlpath', urlpath_workspace)]
    
    # Merge query params into one list of tuples
    query_params_all = query_params_from_hub_url + query_params_for_git
    
    # First build urlencoded query params where the &, =, and / are considered safe. Then, percent encode
    # all characters.
    encoded_query_params = urlencode(query_params_all)
    encoded_query_params_without_safe_chars = quote(urlencode(query_params_all), safe='')
    
    assignment_link_url = urlunparse((domain_scheme, domain_netloc, assignment_link_path, domain_params, encoded_query_params_without_safe_chars, domain_fragment))
    path_url = urlunparse(('', '', path_redirect_url, domain_params, encoded_query_params, domain_fragment))
    
    if is_assignment_link:
        return assignment_link_url
    return path_url

'https://my.hub.com/hub/lti/launch?next=%2Fuser-redirect%2Fgit-pull?repo%3D%26branch%3Dmaster%26urlpath%3Dlab%252Ftree%252F.%252F%253Fautodecode'